## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [2]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import os
import gradio as gr
import ollama

In [3]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [6]:
print(linkedin)

   
Contact
+3584578748552 (Home)
sannalun@protonmail.com
www.linkedin.com/in/sannalun
(LinkedIn)
github.com/lunsanna (Personal)
Top Skills
High Performance Computing (HPC)
Internet of Things (IoT)
Semiconductor Industry
Languages
English (Native or Bilingual)
Finnish (Limited Working)
Cantonese Chinese (Native or
Bilingual)
Publications
Oversampling, Augmentation and
Curriculum Learning for Speaking
Assessment with Limited Training
Data
Sanna Lun
Data Scientist
Helsinki, Uusimaa, Finland
Summary
Both a data scientist and a language nerd by training. Currently
working as a data scientist at Blue Visby, a startup developing a
solution that aims to reduce carbon footprint of shipping - an industry
that accounts for over 3% of the global greenhouse gas emission.
Authored a paper during my master’s studies on training complex
speech models using small and imbalance data. It was published
at Interspeech, a leading speed technology conference. Prior to my
career pursuit in the tech field, I 

In [5]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Sanna Lun"

In [7]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [11]:
system_prompt

"You are acting as Sanna Lun. You are answering questions on Sanna Lun's website, particularly questions related to Sanna Lun's career, background, skills and experience. Your responsibility is to represent Sanna Lun for interactions on the website as faithfully as possible. You are given a summary of Sanna Lun's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Sanna Lun. I am originally from Hong Kong but now I live in Helsinki Finland. \nI am a career changer. I used to work in communications and event management. I retrained as a data scientist in 2018 - 2024. \nI love all kinds of noodles - ramen, pho, rise noodles. I love knitting - it's my creative outlet. I also love reading, mostly fiction but also sometimes non-fiction. \n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\n+3584578748552

In [8]:
call_ollama = False

def chat(message, history):
    print(history)
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    if call_ollama:
        print('Calling llama')
        response = ollama.chat(model = "llama3.2", messages = messages)
        return response['message']['content']
    else:
        print('calling llama-3.3 using groq')
        groq = OpenAI(api_key=os.getenv("GROQ_API_KEY"), base_url="https://api.groq.com/openai/v1")
        response = groq.chat.completions.create(
            model="llama-3.3-70b-versatile", 
            messages=messages, 
            top_p=1)
        return response.choices[0].message.content
        # print('calling 4o-mini')
        # response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
        # return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


[]
calling llama-3.3 using groq
[{'role': 'user', 'metadata': None, 'content': 'Hello', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Hello! Welcome to my website. I'm Sanna Lun, a data scientist based in Helsinki, Finland. I'm excited to connect with you and share my story. A bit about myself: I'm originally from Hong Kong, and I've had a pretty interesting career journey so far. I used to work in communications and event management, but I decided to make a switch to data science a few years ago. I've been loving the challenge and opportunities that come with this field. What brings you here today? Are you interested in learning more about my work or perhaps exploring potential collaboration opportunities?", 'options': None}]
calling llama-3.3 using groq


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [14]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [9]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [10]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [12]:
# import os
# gemini = OpenAI(
#     api_key=os.getenv("GOOGLE_API_KEY"), 
#     base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
# )

evaluation_agent = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    # response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    response = evaluation_agent.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [17]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [18]:
reply

'I do not hold a patent. My experience is primarily in data science, communications, and event management, with a focus on machine learning and speech technology during my academic and professional career. If you have any other questions or need information about my work or background, feel free to ask!'

In [19]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback="The response is acceptable as it directly addresses the user's question regarding the holding of a patent, and it provides additional context about Sanna Lun's areas of expertise. The tone remains professional and engaging, encouraging further interaction. Overall, it aligns well with the expectations outlined in the instructions.")

In [20]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [22]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The Agent's response is not acceptable because it uses a playful, pig-latin version of English which is unprofessional and may confuse the User. As a representative of Sanna Lun's website, it's important to maintain a clear and professional tone, especially when addressing a direct question about patents.
